In [1429]:
# LIBRARIES
import pandas as pd
import numpy as np
from fuzzywuzzy import process
from sklearn.ensemble import IsolationForest

In [1430]:
def read_data(file_name, header=0, sep=','):
    """
    Reads a csv file and returns a pandas dataframe
    """
    return pd.read_csv(file_name, sep=sep, header=header)

In [1431]:
def remove_columns(df, columns):
    """
    Removes columns from a dataframe
    """
    return df.drop(columns, axis=1)

In [1432]:
def remove_rows(df, rows):
    """
    Removes rows from a dataframe
    """
    return df.drop(rows, axis=0)

In [1433]:
def remove_duplicates(df):
    """
    Removes duplicates from a dataframe
    """
    return df.drop_duplicates()

In [1434]:
def convert_to_datatype(df, column, datatype):
    """
    Converts a column to a datatype
    """
    return df[column].astype(datatype)

In [1435]:
def remove_outlier_numeric(df, column, min=-np.inf, max=np.inf):
    """
    Removes rows from a dataframe based on a condition
    """
    if min != -np.inf and max != np.inf:
        return df[(df[column] >= min) & (df[column] <= max)]
    elif min == -np.inf and max != np.inf:
        return df[(df[column] <= max)]
    elif min != -np.inf and max == np.inf:
        return df[(df[column] >= min)]

In [1436]:
def detect_outliers_std(df, column, std=3):
    """
    Detect outliers based on standard deviation
    """
    
    # Set upper and lower limit to 3 standard deviation
    random_data_std = np.std(df[column])
    random_data_mean = np.mean(df[column])
    anomaly_cut_off = random_data_std * std
    
    lower_limit  = random_data_mean - anomaly_cut_off 
    upper_limit = random_data_mean + anomaly_cut_off

    # Generate outliers
    outliers = (df[column] < lower_limit) | (df[column] > upper_limit)
    return outliers

In [1437]:
def detect_outliers_isolation_forest(df, column, contamination=0.1):
    """
    Detect outliers based on isolation forest
    """
    # Create isolation forest
    clf = IsolationForest(random_state=0, contamination=0.1)
    predictions = clf.fit_predict(df[column].to_numpy().reshape(-1, 1))
    return predictions == -1

In [1438]:
def correct_category_levenshtein(df, column, correct_categories, threshold=80):
    """
    Corrects a column by using fuzzywuzzy to find the correct category
    """
    inconsistent_categories = pd.array(list(set(df[column].unique()) - set(correct_categories)))
    
    for inconsistent_category in inconsistent_categories:
        potential_match = process.extractOne(inconsistent_category, correct_categories)
        if potential_match[1] > threshold:
            df.loc[df[column] == inconsistent_category, column] = potential_match[0]
    return df

In [1439]:
def convert_categorical_data(df, column):
    """
    Converts categorical data to numeric data
    """
    return pd.get_dummies(df[column], prefix=column)

In [1440]:
def fill_missing_values(df, column, value):
    """
    Fills missing values in a column with a value
    """
    return df[column].fillna(value)

In [1441]:
def drop_missing_values(df, column):
    """
    Drops missing values in a column
    """
    return df.dropna(subset=[column])

In [1442]:
def knn_impute(df, column):
    """
    Imputation using KNN
    """
    return df[column].fillna(df[column].mode()[0])

In [1443]:
def pearson_spearman_correlation(df, column, output_column):
    """
    Calculates the pearson and spearman correlation between two columns
    """
    return df[column].corr(df[output_column], method='pearson'), df[column].corr(df[output_column], method='spearman')

# TESTING FUNCTIONS

In [1444]:
titanic_df = read_data('../data/raw/titanic/titanic.csv', header=0)

In [1445]:
titanic_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [1446]:
# correct_categories = ['male', 'female']

# titanic_df = correct_category_levenshtein(titanic_df, 'Sex', correct_categories, 80)

In [1454]:
print(pearson_spearman_correlation(titanic_df, 'Fare', 'Survived'))

(0.2573065223849623, 0.32373613944480834)


In [1450]:
# titanic_df[detect_outliers_isolation_forest(titanic_df, 'Fare', 0.1)].shape#.loc[x['Pclass'] == 1, :].shape

(89, 12)